## CSE545-SDG3-Team-Matrix


In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *
import os

In [3]:
sconf = pyspark.SparkConf()
sc = pyspark.SparkContext(appName="SDG3-TeamMatrix")
sqlCtx = SQLContext(sc)
#sconf.set("spark.driver.maxResultSize", "5g")
#sconf.set("spark.driver.memory", "5g")

In [4]:
data_path = r'../LinkPE14US'
#print(os.path.join(data_path, "*.*PUB"))#all
print(os.path.join(data_path, "VS14LINK.USMERGEDPUB"))#*.*PUB
#df = sqlCtx.read.text(os.path.join(data_path, "*.*PUB"))  #all #*.*PUB
df = sqlCtx.read.text(os.path.join(data_path, "VS14LINK.USMERGEDPUB"))  #merged
#df = sqlCtx.read.text(os.path.join(data_path, "VS14LINK.USNUMPUB"))  #350 mb
#df = sqlCtx.read.text("/VS14LINK.USDENPUB") # 5gb
#df = sqlCtx.read.text("/*.*PUB") # both

df.count()


../LinkPE14US/VS14LINK.USMERGEDPUB


26085

In [5]:
def blank_as_Y(x):
    return when(col(x) == '', 'Y').otherwise('N') # 1 == alive, 0 == demised for ICD10

In [20]:
# pandas data frame after adding column metadata
#pndf type = pyspark.sql.dataframe.DataFrame
pndf = df.select(
    df.value.substr(9,4).cast(IntegerType()).alias('Birth_Year'),
    df.value.substr(13,2).cast(IntegerType()).alias('Birth_Month'),
    df.value.substr(19,4).cast(IntegerType()).alias('Time_of_Birth'),
    df.value.substr(23,1).cast(IntegerType()).alias('Birth_DOW'),
    df.value.substr(32,1).cast(IntegerType()).alias('Birth_Place'),
    df.value.substr(50,1).cast(IntegerType()).alias('Facility_Recode'),
    df.value.substr(75,2).cast(IntegerType()).alias('Mothers_Age_Recode'),
    df.value.substr(84,1).cast(IntegerType()).alias('Mothers_Nativity'),
    df.value.substr(104,1).cast(IntegerType()).alias('Residence_Status'),
    df.value.substr(105,2).cast(IntegerType()).alias('Mothers_Race'),
    df.value.substr(110,1).cast(IntegerType()).alias('Bridge_Race_Mother'),
    df.value.substr(115,1).cast(IntegerType()).alias('Mothers_Hispanic_Origin_Recode'),
    df.value.substr(119,1).cast(IntegerType()).alias('Paternity_Ack'),
    df.value.substr(120,1).cast(IntegerType()).alias('Marital_Status'),
    df.value.substr(124,1).cast(IntegerType()).alias('Mothers_Education'),
    df.value.substr(147,2).cast(IntegerType()).alias('Fathers_Age'),
    df.value.substr(149,2).cast(IntegerType()).alias('Fathers_Age_Recode'),
    df.value.substr(151,2).cast(IntegerType()).alias('Fathers_Race_Recode'),
    df.value.substr(156,1).cast(IntegerType()).alias('Bridge_Race_Father'),
    df.value.substr(160,1).cast(IntegerType()).alias('Fathers_Hispanic_Origin_Recode'),
    df.value.substr(163,1).cast(IntegerType()).alias('Fathers_Education'),
    df.value.substr(171,2).cast(IntegerType()).alias('Prior_Births_Living'),
    df.value.substr(173,2).cast(IntegerType()).alias('Prior_Births_Dead'),
    df.value.substr(175,2).cast(IntegerType()).alias('Prior_Other_Terminations'),
    df.value.substr(179,1).cast(IntegerType()).alias('Live_Birth_Order_Recode'),
    df.value.substr(182,1).cast(IntegerType()).alias('Total_Birth_Order_Recode'),
    df.value.substr(198,3).cast(IntegerType()).alias('Interval_Last_Live_Birth_Recode'),
    df.value.substr(201,2).cast(IntegerType()).alias('Interval_Last_Live_Birth_Recode11'),
    df.value.substr(206,3).cast(IntegerType()).alias('Interval_Last_Other_Pregnancy_Recode'),
    df.value.substr(209,2).cast(IntegerType()).alias('Interval_Last_Other_Pregnancy_Recode11'),
    df.value.substr(214,3).cast(IntegerType()).alias('Interval_Last_Pregnancy_Recode'),
    df.value.substr(217,2).cast(IntegerType()).alias('Interval_Last_Pregnancy_Recode11'),
    df.value.substr(238,2).cast(IntegerType()).alias('Num_Prenatal_Visits'),
    df.value.substr(242,2).cast(IntegerType()).alias('Num_Prenatal_Visits_Recode'),
    df.value.substr(253,2).cast(IntegerType()).alias('Cigarettes_Before_Pregnancy'),
    df.value.substr(255,2).cast(IntegerType()).alias('Cigarettes_1st_Trimester'),
    df.value.substr(257,2).cast(IntegerType()).alias('Cigarettes_2nd_Trimester'),
    df.value.substr(259,2).cast(IntegerType()).alias('Cigarettes_3rd_Trimester'),
    df.value.substr(261,1).cast(IntegerType()).alias('Cigarettes_Before_Pregnancy_Recode'),
    df.value.substr(262,1).cast(IntegerType()).alias('Cigarettes_1st_Trimester_Recode'),
    df.value.substr(263,1).cast(IntegerType()).alias('Cigarettes_2nd_Trimester_Recode'),
    df.value.substr(264,1).cast(IntegerType()).alias('Cigarettes_3rd_Trimester_Recode'),
    df.value.substr(280,2).cast(IntegerType()).alias('Mothers_Height_Inches'),
    df.value.substr(287,1).cast(IntegerType()).alias('BMI_Recode'),
    df.value.substr(292,3).cast(IntegerType()).alias('Pre_Pregnancy_Weight_Recode'),
    df.value.substr(299,3).cast(IntegerType()).alias('Delivery_Weight_Recode'),
    df.value.substr(304,2).cast(IntegerType()).alias('Weight_Gain'),
    df.value.substr(306,1).cast(IntegerType()).alias('Weight_Gain_Recode'),
    df.value.substr(313,1).cast(StringType()).alias('Pre_Pregnancy_Diabetes'),
    df.value.substr(314,1).cast(StringType()).alias('Gestational_Diabetes'),
    df.value.substr(315,1).cast(StringType()).alias('Pre_Pregnancy_Hypertension'),
    df.value.substr(316,1).cast(StringType()).alias('Gestational_Hypertension'),
    df.value.substr(317,1).cast(StringType()).alias('Hypertension_Eclmapsia'),
    df.value.substr(318,1).cast(StringType()).alias('Previous_Preterm_Birth'),
    df.value.substr(325,1).cast(StringType()).alias('Infertility_Treatment'),
    df.value.substr(326,1).cast(StringType()).alias('Fertility_Enhancing_Drugs'),
    df.value.substr(327,1).cast(StringType()).alias('Assistive_Reproductive_Technology'),
    df.value.substr(331,1).cast(StringType()).alias('Prev_Cesareans'),
    df.value.substr(332,2).cast(IntegerType()).alias('Num_Prev_Cesareans'),
    df.value.substr(337,1).cast(StringType()).alias('No_Risk_Factors_Checked'),
    
    df.value.substr(343,1).cast(StringType()).alias('Gonorrhea'),
    df.value.substr(344,1).cast(StringType()).alias('Syphillis'),
    df.value.substr(345,1).cast(StringType()).alias('Chlamydia'),
    df.value.substr(346,1).cast(StringType()).alias('HepatitisB'),
    df.value.substr(347,1).cast(StringType()).alias('HepatitisC'),
    df.value.substr(353,1).cast(StringType()).alias('No_Infections_Checked'),
    df.value.substr(360,1).cast(StringType()).alias('Sucessful_External_Cephalic_Version'),
    df.value.substr(361,1).cast(StringType()).alias('Failed_External_Cephalic_Version'),
    
    df.value.substr(383,1).cast(StringType()).alias('Induction_Of_Labor'),
    df.value.substr(384,1).cast(StringType()).alias('Augmentation_Of_Labor'),
    df.value.substr(385,1).cast(StringType()).alias('Steroids_Mother'),
    df.value.substr(386,1).cast(StringType()).alias('Antibiotics_Mother'),
    df.value.substr(386,1).cast(StringType()).alias('Chorioamnionitis_Mother'),
    df.value.substr(387,1).cast(StringType()).alias('Anesthesia_Mother'),
    df.value.substr(395,1).cast(StringType()).alias('No_Labor_Checked'),
    df.value.substr(401,1).cast(IntegerType()).alias('Fetal_Presentation'),
    df.value.substr(402,1).cast(IntegerType()).alias('Final_Route_and_Method_Of_Delivery'),
    df.value.substr(403,1).cast(IntegerType()).alias('Trail_Of_Labor_Attempted'),
    df.value.substr(407,1).cast(IntegerType()).alias('Delivery_Method_Recode'),
    df.value.substr(415,1).cast(StringType()).alias('Maternal_Transfusion'),
    df.value.substr(416,1).cast(StringType()).alias('Perineal_Laceration'),
    df.value.substr(417,1).cast(StringType()).alias('Ruptured_Uterus'),
    df.value.substr(418,1).cast(StringType()).alias('Unplanned_Hysterectomy'),
    df.value.substr(419,1).cast(StringType()).alias('Admit_to_Intensive_Care'),
    df.value.substr(427,1).cast(IntegerType()).alias('No_Maternal_Morbidity_Checked'),
    df.value.substr(433,1).cast(IntegerType()).alias('Attendant'),
    df.value.substr(434,1).cast(StringType()).alias('Mother_Transferred'),

    
    #435 - 870
    df.value.substr(435,1).cast(IntegerType()).alias('Payment_Source'),
    df.value.substr(436,1).cast(IntegerType()).alias('Payment_Recode'),
    df.value.substr(444,2).cast(IntegerType()).alias('Five_Minute_APGAR_Score'),
    df.value.substr(446,1).cast(IntegerType()).alias('Five_Minute_APGAR_Recode'),
    df.value.substr(448,2).cast(IntegerType()).alias('Ten_Minute_APGAR_Score'),
    df.value.substr(450,1).cast(IntegerType()).alias('Ten_Minute_APGAR_Recode'),
    df.value.substr(454,1).cast(IntegerType()).alias('Plurality'),
    df.value.substr(475,1).cast(StringType()).alias('Sex_Of_Infant'),
    df.value.substr(477,2).cast(IntegerType()).alias('Last_Normal_Menses_Month'),
    df.value.substr(481,4).cast(IntegerType()).alias('Last_Normal_Menses_Year'),
    df.value.substr(489,1).cast(IntegerType()).alias('Obstetric_Estimate_of_Gestation_Used_Flag'),
    df.value.substr(490,2).cast(IntegerType()).alias('Combined_Gestation_Detail_in_Weeks'),
    df.value.substr(492,1).cast(IntegerType()).alias('Combined_Gestation_Recode_Weeks'),
    df.value.substr(509,2).cast(IntegerType()).alias('Birth_Weight_Recode_14'),
    df.value.substr(511,1).cast(IntegerType()).alias('Birth_Weight_Recode_4'),
    df.value.substr(512,4).cast(IntegerType()).alias('Imputed_Birthwieght'),
    #Abnormal Conditions of the Newborn
    df.value.substr(517,1).cast(StringType()).alias('Assisted_Ventilation'),
    df.value.substr(518,1).cast(StringType()).alias('Assisted_Ventilation_Greater_Than_6_Hours'),
    df.value.substr(519,1).cast(StringType()).alias('Admission_to_NICU'),
    df.value.substr(520,1).cast(StringType()).alias('Surfactant'),
    df.value.substr(521,1).cast(StringType()).alias('Antibiotics'),
    df.value.substr(522,1).cast(StringType()).alias('Seizures'),
    df.value.substr(531,1).cast(IntegerType()).alias('No_Abnormal_Conditions_Checked'),
    #Congenital Anomalies of the Newborn
    df.value.substr(537,1).cast(StringType()).alias('Anencephaly'),
    df.value.substr(538,1).cast(StringType()).alias('Meningomyelocele_or_Spina_Bifida'),
    df.value.substr(539,1).cast(StringType()).alias('Cyanotic_Congenital_Heart_Disease'),
    df.value.substr(540,1).cast(StringType()).alias('Congenital_Diaphragmatic_Hernia'),
    df.value.substr(541,1).cast(StringType()).alias('Omphalocele'),
    df.value.substr(542,1).cast(StringType()).alias('Gastroschisis'),
    df.value.substr(549,1).cast(StringType()).alias('Limb_Reduction_Defect'),
    df.value.substr(550,1).cast(StringType()).alias('Cleft_Lip_w_or_wo_Cleft_Palate'),
    df.value.substr(551,1).cast(StringType()).alias('Cleft_Palate_alone'),
    df.value.substr(553,1).cast(StringType()).alias('Suspected_Chromosomal_Disorder'),
    df.value.substr(554,1).cast(StringType()).alias('Hypospadias'),
    df.value.substr(561,1).cast(IntegerType()).alias('No_Congenital_Anomalies_Checked'),
    df.value.substr(568,1).cast(StringType()).alias('Infant_Living_at_Time_of_Report'),
    df.value.substr(569,1).cast(StringType()).alias('Infant_Being_Breastfed'), 
    #
    #1330 - 1743
    df.value.substr(1331,1).cast(IntegerType()).alias('Diabetes'),
    df.value.substr(1332,1).cast(IntegerType()).alias('Chronic_Hypertension'),
    df.value.substr(1333,1).cast(IntegerType()).alias('Prepregnacy_Associated_Hypertension'),
    df.value.substr(1334,1).cast(IntegerType()).alias('Eclampsia'),
    df.value.substr(1335,1).cast(IntegerType()).alias('Forceps'),
    df.value.substr(1336,1).cast(IntegerType()).alias('Vacuum'),
    df.value.substr(1337,1).cast(IntegerType()).alias('Induction_of_Labor1'),
    df.value.substr(1338,1).cast(IntegerType()).alias('Breech1'),
    df.value.substr(1341,1).cast(IntegerType()).alias('Spina_Bifida_Meningocele1'),
    df.value.substr(1342,1).cast(IntegerType()).alias('Omphalocele_Gastroschisis1'),
    
    df.value.substr(1343,1).cast(IntegerType()).alias('Cleft_Lip_Palate1'), 
    df.value.substr(1344,1).cast(IntegerType()).alias('Diaphragmatic_Herni1a'),
    df.value.substr(1345,1).cast(IntegerType()).alias('Down_Syndrome1'),
    df.value.substr(1356,3).cast(IntegerType()).alias('Age_at_Death_in_Days'),
    df.value.substr(1359,1).cast(IntegerType()).alias('Infant_age_recode_5'),
    df.value.substr(1360,2).cast(IntegerType()).alias('Infant_age_recode_22'), 
    df.value.substr(1362,1).cast(IntegerType()).alias('Manner_of_Death'),
    
    df.value.substr(1366,1).cast(IntegerType()).alias('Place_of_injury_for_causes_W00_Y34_except_Y06'),
    df.value.substr(1368,4).cast(StringType()).alias('ICD10'), #underlying cause of death - this is used to determine the LABEL for ML
    df.value.substr(1671,1).cast(IntegerType()).alias('Day_of_Week_of_Death'),
    df.value.substr(1672,4).cast(IntegerType()).alias('Data_Year'),
    df.value.substr(1742,1).cast(IntegerType()).alias('Month_of_Death')
    
    
).withColumn("id", monotonically_increasing_id())

pndf.printSchema
pndf = pndf.na.fill({'Num_Prev_Cesareans': 0.0})

# derive the infant_living column from ICD10 - must for ML prediction
pndf = pndf.withColumn("Infant_Living", blank_as_Y("ICD10")) # dervied 0/1 label  # 1 == alive, 0 == demised for ICD10
print("number of features ", len(df.columns))

number of features  1


In [21]:
print(pndf.filter(pndf['Infant_Living_at_Time_of_Report']!='Y').count())
pndf.describe(['ICD10']).show()
print("# Infants Living >> ",pndf.filter(pndf['Infant_Living']=='Y').count())

11351
+-------+-----+
|summary|ICD10|
+-------+-----+
|  count|26085|
|   mean| null|
| stddev| null|
|    min|     |
|    max| Y848|
+-------+-----+

# Infants Living >>  3000


In [22]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

print("Before cleanup count of PNDF ",pndf.count())
pndf = pndf.na.fill("NA") #pndf.na.fill("0.0") pndf.na.drop()
print("After fill count of PNDF ",pndf.count())

# String Indexer .setHandleInvalid("keep")
stringColumns = ['Pre_Pregnancy_Diabetes','Gestational_Diabetes','Pre_Pregnancy_Hypertension', 'Gestational_Hypertension',\
                'Hypertension_Eclmapsia','Previous_Preterm_Birth','Infertility_Treatment','Fertility_Enhancing_Drugs',\
                'Assistive_Reproductive_Technology', 'Prev_Cesareans','No_Risk_Factors_Checked','Sex_Of_Infant','Assisted_Ventilation',\
                'Assisted_Ventilation_Greater_Than_6_Hours', 'Admission_to_NICU','Surfactant','Antibiotics','Seizures','Anencephaly',\
                'Meningomyelocele_or_Spina_Bifida','Cyanotic_Congenital_Heart_Disease','Congenital_Diaphragmatic_Hernia',\
                'Omphalocele','Gastroschisis','Limb_Reduction_Defect','Cleft_Lip_w_or_wo_Cleft_Palate','Cleft_Palate_alone',\
                'Suspected_Chromosomal_Disorder','Hypospadias','Infant_Living_at_Time_of_Report','Infant_Being_Breastfed',\
                 'Gonorrhea','Syphillis','Chlamydia','HepatitisB','HepatitisC','No_Infections_Checked','Sucessful_External_Cephalic_Version',\
                 'Failed_External_Cephalic_Version','Induction_Of_Labor','Augmentation_Of_Labor','Steroids_Mother','Antibiotics_Mother','Chorioamnionitis_Mother',\
                 'Anesthesia_Mother','No_Labor_Checked','Maternal_Transfusion','Perineal_Laceration','Ruptured_Uterus','Unplanned_Hysterectomy',\
                 'Admit_to_Intensive_Care','Mother_Transferred','ICD10', 'Infant_Living']
indexers = [StringIndexer(inputCol=column, outputCol=column+"_numeric").fit(pndf) for column in list(  set(stringColumns)) ]
label_stringIndexer = StringIndexer(inputCol="Infant_Living", outputCol='label') # TODO Change to 'Infant_Living' after fixing cleanup
indexers.append(label_stringIndexer)

pipeline = Pipeline(stages=indexers)
pndf = pipeline.fit(pndf).transform(pndf)
print("After cleanup count of PNDF ",pndf.count())
pndf.head(1)


Before cleanup count of PNDF  26085
After fill count of PNDF  26085
After cleanup count of PNDF  26085


[Row(Birth_Year=2013, Birth_Month=7, Time_of_Birth=1853, Birth_DOW=3, Birth_Place=1, Facility_Recode=1, Mothers_Age_Recode=24, Mothers_Nativity=1, Residence_Status=1, Mothers_Race=None, Bridge_Race_Mother=1, Mothers_Hispanic_Origin_Recode=0, Paternity_Ack=None, Marital_Status=1, Mothers_Education=4, Fathers_Age=22, Fathers_Age_Recode=3, Fathers_Race_Recode=None, Bridge_Race_Father=1, Fathers_Hispanic_Origin_Recode=0, Fathers_Education=2, Prior_Births_Living=0, Prior_Births_Dead=0, Prior_Other_Terminations=0, Live_Birth_Order_Recode=1, Total_Birth_Order_Recode=1, Interval_Last_Live_Birth_Recode=888, Interval_Last_Live_Birth_Recode11=88, Interval_Last_Other_Pregnancy_Recode=888, Interval_Last_Other_Pregnancy_Recode11=88, Interval_Last_Pregnancy_Recode=888, Interval_Last_Pregnancy_Recode11=88, Num_Prenatal_Visits=8, Num_Prenatal_Visits_Recode=5, Cigarettes_Before_Pregnancy=20, Cigarettes_1st_Trimester=3, Cigarettes_2nd_Trimester=1, Cigarettes_3rd_Trimester=1, Cigarettes_Before_Pregnancy_R

In [23]:
from pyspark.ml.feature import VectorAssembler

pca_cols = pndf.columns
pca_cols = [e for e in pca_cols if e not in stringColumns]
no_label_pca_cols = [e for e in pca_cols if e not in ['Infant_Living_at_Time_of_Report_numeric','Infant_Living_at_Time_of_Report',\
                                                    'Infant_Living','Infant_Living_numeric','label']] #removed labels

assembler = VectorAssembler(inputCols=no_label_pca_cols, outputCol='features' )
vector_df = assembler.transform(pndf.na.fill(-1.0)) #pndf.na.drop()
vector_df.head()

Row(Birth_Year=2013, Birth_Month=7, Time_of_Birth=1853, Birth_DOW=3, Birth_Place=1, Facility_Recode=1, Mothers_Age_Recode=24, Mothers_Nativity=1, Residence_Status=1, Mothers_Race=-1, Bridge_Race_Mother=1, Mothers_Hispanic_Origin_Recode=0, Paternity_Ack=-1, Marital_Status=1, Mothers_Education=4, Fathers_Age=22, Fathers_Age_Recode=3, Fathers_Race_Recode=-1, Bridge_Race_Father=1, Fathers_Hispanic_Origin_Recode=0, Fathers_Education=2, Prior_Births_Living=0, Prior_Births_Dead=0, Prior_Other_Terminations=0, Live_Birth_Order_Recode=1, Total_Birth_Order_Recode=1, Interval_Last_Live_Birth_Recode=888, Interval_Last_Live_Birth_Recode11=88, Interval_Last_Other_Pregnancy_Recode=888, Interval_Last_Other_Pregnancy_Recode11=88, Interval_Last_Pregnancy_Recode=888, Interval_Last_Pregnancy_Recode11=88, Num_Prenatal_Visits=8, Num_Prenatal_Visits_Recode=5, Cigarettes_Before_Pregnancy=20, Cigarettes_1st_Trimester=3, Cigarettes_2nd_Trimester=1, Cigarettes_3rd_Trimester=1, Cigarettes_Before_Pregnancy_Recode=-

### Scaling and Normalization

In [24]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features', 
                        outputCol='scaledFeatures', withMean=False, withStd=True) # TODO: should withMean be True??
scaled_df = scaler.fit(vector_df).transform(vector_df)
scaled_df.head()

Row(Birth_Year=2013, Birth_Month=7, Time_of_Birth=1853, Birth_DOW=3, Birth_Place=1, Facility_Recode=1, Mothers_Age_Recode=24, Mothers_Nativity=1, Residence_Status=1, Mothers_Race=-1, Bridge_Race_Mother=1, Mothers_Hispanic_Origin_Recode=0, Paternity_Ack=-1, Marital_Status=1, Mothers_Education=4, Fathers_Age=22, Fathers_Age_Recode=3, Fathers_Race_Recode=-1, Bridge_Race_Father=1, Fathers_Hispanic_Origin_Recode=0, Fathers_Education=2, Prior_Births_Living=0, Prior_Births_Dead=0, Prior_Other_Terminations=0, Live_Birth_Order_Recode=1, Total_Birth_Order_Recode=1, Interval_Last_Live_Birth_Recode=888, Interval_Last_Live_Birth_Recode11=88, Interval_Last_Other_Pregnancy_Recode=888, Interval_Last_Other_Pregnancy_Recode11=88, Interval_Last_Pregnancy_Recode=888, Interval_Last_Pregnancy_Recode11=88, Num_Prenatal_Visits=8, Num_Prenatal_Visits_Recode=5, Cigarettes_Before_Pregnancy=20, Cigarettes_1st_Trimester=3, Cigarettes_2nd_Trimester=1, Cigarettes_3rd_Trimester=1, Cigarettes_Before_Pregnancy_Recode=-

In [14]:
from pyspark.ml.feature import Normalizer
nrmlzer = Normalizer(inputCol='scaledFeatures', outputCol='normalizeFeatures', p=1.0)
l1Normalized = nrmlzer.transform(scaled_df)
l1Normalized.head()

Row(Birth_Year=2013, Birth_Month=7, Time_of_Birth=1853, Birth_DOW=3, Birth_Place=1, Facility_Recode=1, Mothers_Age_Recode=24, Mothers_Nativity=1, Residence_Status=1, Mothers_Race=-1, Bridge_Race_Mother=1, Mothers_Hispanic_Origin_Recode=0, Paternity_Ack=-1, Marital_Status=1, Mothers_Education=4, Fathers_Age=22, Fathers_Age_Recode=3, Fathers_Race_Recode=-1, Bridge_Race_Father=1, Fathers_Hispanic_Origin_Recode=0, Fathers_Education=2, Prior_Births_Living=0, Prior_Births_Dead=0, Prior_Other_Terminations=0, Live_Birth_Order_Recode=1, Total_Birth_Order_Recode=1, Interval_Last_Live_Birth_Recode=888, Interval_Last_Live_Birth_Recode11=88, Interval_Last_Other_Pregnancy_Recode=888, Interval_Last_Other_Pregnancy_Recode11=88, Interval_Last_Pregnancy_Recode=888, Interval_Last_Pregnancy_Recode11=88, Num_Prenatal_Visits=8, Num_Prenatal_Visits_Recode=5, Cigarettes_Before_Pregnancy=20, Cigarettes_1st_Trimester=3, Cigarettes_2nd_Trimester=1, Cigarettes_3rd_Trimester=1, Cigarettes_Before_Pregnancy_Recode=-

### Dimensionality Reduction: PCA and Similarity Search(Cosine similarity using DIMSUM)

In [33]:
from pyspark.ml.feature import PCA
from pyspark.mllib.linalg.distributed import RowMatrix
num_principal_comp = 50
pca = PCA(k=num_principal_comp, inputCol='normalizeFeatures', outputCol='features_pca')
pca_model = pca.fit(l1Normalized)
pca_feat = pca_model.transform(l1Normalized)
print(type(pca_feat))
# print(pca_feat.select('features_pca').first())
# rddVector = pca_feat.select('features_pca').rdd.map(lambda row : list(row[0]))
# mat = RowMatrix(rddVector)
# exact = mat.columnSimilarities()
# # # print(exact.entries.collect())
# print(exact.entries.count())

<class 'pyspark.sql.dataframe.DataFrame'>
